In [7]:
import numpy
import os
import xarray
from datetime import datetime, timedelta

In [8]:
MOD09GA_REFLECTANCE_FLODER = '/disk2/Data/MOD09GA_b1b2_JP'

In [9]:
def read_mod09ga_site(modis_mod09_tif, lat_site, lon_site):
    modis_ds = xarray.open_rasterio(modis_mod09_tif)[0]
    jp_ds = modis_ds.interp(x=[lon_site], y=[lat_site], method="nearest", kwargs={"fill_value": "extrapolate"}) # linear
    r_dn = numpy.array(jp_ds.values)[0][0]
    if r_dn == -28672:
        return numpy.NaN
    else:
        data_v = r_dn*0.0001
        return data_v

In [10]:
tky_site_loc = (36.146, 137.423) # lat, lon
crk_site_loc = (38.201, 127.251) # lat, lon
gck_site_loc = (37.748, 127.162) # lat, lon
gdk_site_loc = (37.749, 127.149) # lat, lon
site_names = ['TKY','CRK', 'GCK', 'GDK']
site_locs = [tky_site_loc, crk_site_loc, gck_site_loc, gdk_site_loc]

In [11]:
years = ['2018', '2019']
year_days = [(3-len(str(day_n)))*'0' + str(day_n) for day_n in numpy.arange(1, 366, 1)]

for current_site_idx in range(len(site_names)):
    current_site_loc = site_locs[current_site_idx]
    
    site_record = []
    for year_str in years:
        year_first_day_date = datetime.strptime(year_str + '-01-01T00:00:00Z', "%Y-%m-%dT%H:%M:%SZ")
        for day_idx in range(len(year_days)):
            day_str = year_days[day_idx]
            data_filename1 = os.path.join(MOD09GA_REFLECTANCE_FLODER, year_str, 'MOD09GA.061_sur_refl_b01_1_doy' + year_str + day_str + '_aid0001.tif')
            data_filename2 = os.path.join(MOD09GA_REFLECTANCE_FLODER, year_str, 'MOD09GA.061_sur_refl_b02_1_doy' + year_str + day_str + '_aid0001.tif')

            this_day_date = year_first_day_date + timedelta(days=day_idx)
            this_day_time = this_day_date.strftime("%Y-%m-%dT%H:%M:%SZ")

            day_site_v1 = read_mod09ga_site(data_filename1, current_site_loc[0], current_site_loc[1])
            day_site_v2 = read_mod09ga_site(data_filename2, current_site_loc[0], current_site_loc[1])
            record_item = [this_day_time, str(day_site_v1), str(day_site_v2)]

            site_record.append(record_item)
    modis_record = numpy.array(site_record)
    numpy.savetxt(os.path.join(MOD09GA_REFLECTANCE_FLODER, site_names[current_site_idx]+'_MOD09GA_ReflectanceB1B2'+'.csv'), modis_record, delimiter=",", fmt='%s')